In [1]:
import os, sys
sys.path.append('..')
import common, sql
import pandas as pd

In [11]:
q='''select distinct v.id, severity, CVSS3_severity from npmAlert nA
    join vulnerability v on nA.vulnerabilityId = v.id
    where toolId=5
    and cveId is not null;'''
df=sql.pd_read_sql(q)
df.loc[(df.severity=='low'),'severity']=1
df.loc[(df.severity=='moderate'),'severity']=2
df.loc[(df.severity=='high'),'severity']=3
df.loc[(df.severity=='critical'),'severity']=4
df.loc[df['CVSS3_severity']=='MEDIUM','CVSS3_severity']=2
df.loc[df['CVSS3_severity']=='HIGH','CVSS3_severity']=3
df.loc[df['CVSS3_severity']=='CRITICAL','CVSS3_severity']=4
df = df[df['CVSS3_severity'].notna()]
df

,id,severity,CVSS3_severity
0,151,1,2
1,143,1,2
2,141,3,2
3,137,1,4
4,153,2,3
5,145,2,2
6,1208,1,2
7,155,3,3
8,207,1,2
9,973,1,2


In [16]:
df['d'] = df['CVSS3_severity'] - df.severity
df

,id,severity,CVSS3_severity,diff,d
0,151,1,2,1,1
1,143,1,2,1,1
2,141,3,2,-1,-1
3,137,1,4,3,3
4,153,2,3,1,1
5,145,2,2,0,0
6,1208,1,2,1,1
7,155,3,3,0,0
8,207,1,2,1,1
9,973,1,2,1,1


In [17]:
df.d.median()

0.5

In [22]:
dfs = df.groupby('d')
for name, group in dfs:
    print(name, len(group))

-1 2
0 13
1 13
2 1
3 1


In [27]:
q='''select distinct v.id, severity, CVSS2_severity, CVSS3_severity from npmAlert nA
join vulnerability v on nA.vulnerabilityId = v.id
where toolId=3
and cveId is not null
union
select distinct v.id, severity, CVSS2_severity, CVSS3_severity from mavenAlert nA
join vulnerability v on nA.vulnerabilityId = v.id
where toolId=3
and cveId is not null;'''
df=sql.pd_read_sql(q)
df.loc[(df.severity=='LOW'),'severity']=1
df.loc[(df.severity=='MODERATE'),'severity']=2
df.loc[(df.severity=='HIGH'),'severity']=3
df.loc[(df.severity=='CRITICAL'),'severity']=4
df.loc[df['CVSS3_severity']=='LOW','CVSS3_severity']=1
df.loc[df['CVSS3_severity']=='MEDIUM','CVSS3_severity']=2
df.loc[df['CVSS3_severity']=='HIGH','CVSS3_severity']=3
df.loc[df['CVSS3_severity']=='CRITICAL','CVSS3_severity']=4
df = df[df['CVSS3_severity'].notna()]
df

,id,severity,CVSS2_severity,CVSS3_severity
0,283,2,MEDIUM,3
1,153,2,MEDIUM,3
2,1056,3,MEDIUM,3
3,1057,3,MEDIUM,3
4,155,3,MEDIUM,3
...,...,...,...,...
85,98,4,HIGH,4
86,99,3,MEDIUM,3
87,34,3,MEDIUM,3
88,310,2,MEDIUM,2


In [28]:
df['d'] = df['CVSS3_severity'] - df.severity
dfs = df.groupby('d')
for name, group in dfs:
    print(name, len(group))

-1 2
0 30
1 42
2 12
